In [ ]:
#OK we are going to combine odds and fights

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
event_file = 'scraper_helpers/scraped_event_with_ranks.csv'

In [3]:
print("IS UPCOMING?")
print(is_upcoming)

IS UPCOMING?


NameError: name 'is_upcoming' is not defined

In [ ]:
df = pd.read_csv(event_file)

In [ ]:
d = df['date'].iloc[0]

In [ ]:
d

In [ ]:
odds_file = 'odds_pages/'+str(d)+'.html'

In [ ]:
odds_file

In [ ]:
#is_most_recent = True
#is_upcoming = False
#upcoming_odds_page = "https://www.bestfightodds.com/events/ufc-on-espn-25-korean-zombie-vs-ige-2130"


In [ ]:
if is_upcoming or is_most_recent:
    html = urlopen(upcoming_odds_page)
else:
    html = open(odds_file, encoding='utf-8')

In [ ]:
bs = BeautifulSoup(html.read(), 'html.parser')

In [ ]:
#Let's grab the odds table?
odds_table = bs.find_all('table', {'class':'odds-table'})

In [ ]:
len(odds_table)

In [ ]:
odds_table[1]

In [ ]:
all_rows = odds_table[1].find_all('tr')

In [ ]:
#OK..... Let's get 2 lists.  Red fighters, and blue fighters
red_fighter_list = df['R_fighter'].values
blue_fighter_list = df['B_fighter'].values

In [ ]:
red_odds_list = [None] * len(red_fighter_list)
blue_odds_list = [None] * len(red_fighter_list)

In [ ]:
#all_rows[1]

In [ ]:
for n in range(len(red_fighter_list)):
    #print(red_fighter_list[n])
    for r in all_rows:
        if str(r).find(str(red_fighter_list[n])) > -1:
            print(r)
            #OK.... It looks like it's going to be the 2nd span....
            temp_odds = r.find_all('td')[2].find('span')
            if temp_odds is not None:
                temp_odds = temp_odds.get_text().strip()
            red_odds_list[n] = (temp_odds)
            break
            
for n in range(len(blue_fighter_list)):
    #print(red_fighter_list[n])
    for r in all_rows:
        if str(r).find(str(blue_fighter_list[n])) > -1:
            #OK.... It looks like it's going to be the 2nd span....
            temp_odds = r.find_all('td')[2].find('span')
            if temp_odds is not None:
                temp_odds = temp_odds.get_text().strip()
            blue_odds_list[n] = (temp_odds)
            break


In [ ]:
r_ev_list = []
b_ev_list = []
for r in red_odds_list:
    if r is not None:
        if int(r) > 0:
            r_ev_list.append(r)
        else:
            r_ev_list.append(-(100/int(r)) * 100)
    else:
        r_ev_list.append(None)
        
for b in blue_odds_list:
    if b is not None: 
        if int(b) > 0:
            b_ev_list.append(b)
        else:
            b_ev_list.append(-(100/int(b)) * 100)       
    else:
        b_ev_list.append(None)

In [ ]:
print(b_ev_list)

In [ ]:
total_odds_list = zip(red_fighter_list, blue_fighter_list, red_odds_list, blue_odds_list, r_ev_list, b_ev_list)

In [ ]:
for r_fighter, b_fighter, r_odds, b_odds, r_ev, b_ev in total_odds_list:
    print(r_odds)
    df.loc[df.R_fighter == r_fighter, ['R_odds', 'B_odds', 'R_ev', 'B_ev']] = (r_odds), (b_odds), (r_ev), (b_ev)
    


In [ ]:
df

In [ ]:
master_df = pd.read_csv('../data/kaggle_data/ufc-master.csv')

In [ ]:
master_df

In [ ]:
df

In [ ]:
if is_upcoming:
    df['Winner'] = ''
    #df.to_csv('upcoming-event.csv' , index=False)
    df.to_csv('../data/kaggle_data/upcoming-event.csv', index=False)
    print("saved upcoming event")
elif is_most_recent:
    df.to_csv('../data/kaggle_data/most-recent-event.csv' , index=False)   
    temp_df = df.append(master_df)
    #temp_df.to_csv('ufc-master.csv', index=False)
    temp_df.to_csv('../data/kaggle_data/ufc-master.csv', index=False)    
else:
    temp_df = df.append(master_df)
    #temp_df.to_csv('ufc-master.csv', index=False)
    temp_df.to_csv('../data/kaggle_data/ufc-master.csv', index=False)

In [ ]:
temp_df